In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/march-machine-learning-mania-2024/Conferences.csv
/kaggle/input/march-machine-learning-mania-2024/sample_submission.csv
/kaggle/input/march-machine-learning-mania-2024/WNCAATourneyDetailedResults.csv
/kaggle/input/march-machine-learning-mania-2024/WRegularSeasonCompactResults.csv
/kaggle/input/march-machine-learning-mania-2024/MNCAATourneySeedRoundSlots.csv
/kaggle/input/march-machine-learning-mania-2024/MRegularSeasonDetailedResults.csv
/kaggle/input/march-machine-learning-mania-2024/MNCAATourneyCompactResults.csv
/kaggle/input/march-machine-learning-mania-2024/MGameCities.csv
/kaggle/input/march-machine-learning-mania-2024/WGameCities.csv
/kaggle/input/march-machine-learning-mania-2024/MSeasons.csv
/kaggle/input/march-machine-learning-mania-2024/WNCAATourneySlots.csv
/kaggle/input/march-machine-learning-mania-2024/MSecondaryTourneyTeams.csv
/kaggle/input/march-machine-learning-mania-2024/2024_tourney_seeds.csv
/kaggle/input/march-machine-learning-mania-2024/Cities.csv
/

In [2]:
import polars as pl
import os

TABS = {}
for file in os.listdir('../input/march-machine-learning-mania-2024/'):
    try:
        TABS[file[:-4]] = pl.read_csv(f'../input/march-machine-learning-mania-2024/{file}')
    except pl.ComputeError as e:
        print(f"Could not read file {file}")

Could not read file MTeamSpellings.csv
Could not read file WTeamSpellings.csv


In [3]:
def seeds_to_dict(seeds):
    return {row['TeamID']: row['Seed'] for row in seeds.iter_rows(named=True)}


def slots_to_dict(slots):
    return {row['Slot']: (row['StrongSeed'], row['WeakSeed']) 
            for row in slots.iter_rows(named=True)}


def results_to_dict(results_tab, seeds_dict):
    out = {}
    for row in results_tab.iter_rows(named=True):
        wid = seeds_dict[row['WTeamID']]
        lid = seeds_dict[row['LTeamID']]
        out[(wid, lid)] = wid
        out[(lid, wid)] = wid
    return out


LEVELS = [
    ['R1W1', 'R1W2', 'R1W3', 'R1W4', 'R1W5', 'R1W6', 'R1W7', 'R1W8', 'R1X1', 'R1X2', 'R1X3', 'R1X4', 'R1X5', 'R1X6', 'R1X7', 'R1X8', 'R1Y1', 'R1Y2', 'R1Y3', 'R1Y4', 'R1Y5', 'R1Y6', 'R1Y7', 'R1Y8', 'R1Z1', 'R1Z2', 'R1Z3', 'R1Z4', 'R1Z5', 'R1Z6', 'R1Z7', 'R1Z8'], 
    ['R2W1', 'R2W2', 'R2W3', 'R2W4', 'R2X1', 'R2X2', 'R2X3', 'R2X4', 'R2Y1', 'R2Y2', 'R2Y3', 'R2Y4', 'R2Z1', 'R2Z2', 'R2Z3', 'R2Z4'],
    ['R3W1', 'R3W2', 'R3X1', 'R3X2', 'R3Y1', 'R3Y2', 'R3Z1', 'R3Z2'], 
    ['R4W1', 'R4X1', 'R4Y1', 'R4Z1'], 
    ['R5WX', 'R5YZ'], 
    ['R6CH'],
]

SLOTS = {'R1W1': ('W01', 'W16'), 'R1W2': ('W02', 'W15'), 'R1W3': ('W03', 'W14'), 'R1W4': ('W04', 'W13'), 'R1W5': ('W05', 'W12'), 'R1W6': ('W06', 'W11'), 'R1W7': ('W07', 'W10'), 'R1W8': ('W08', 'W09'), 'R1X1': ('X01', 'X16'), 'R1X2': ('X02', 'X15'), 'R1X3': ('X03', 'X14'), 'R1X4': ('X04', 'X13'), 'R1X5': ('X05', 'X12'), 'R1X6': ('X06', 'X11'), 'R1X7': ('X07', 'X10'), 'R1X8': ('X08', 'X09'), 'R1Y1': ('Y01', 'Y16'), 'R1Y2': ('Y02', 'Y15'), 'R1Y3': ('Y03', 'Y14'), 'R1Y4': ('Y04', 'Y13'), 'R1Y5': ('Y05', 'Y12'), 'R1Y6': ('Y06', 'Y11'), 'R1Y7': ('Y07', 'Y10'), 'R1Y8': ('Y08', 'Y09'), 'R1Z1': ('Z01', 'Z16'), 'R1Z2': ('Z02', 'Z15'), 'R1Z3': ('Z03', 'Z14'), 'R1Z4': ('Z04', 'Z13'), 'R1Z5': ('Z05', 'Z12'), 'R1Z6': ('Z06', 'Z11'), 'R1Z7': ('Z07', 'Z10'), 'R1Z8': ('Z08', 'Z09'), 'R2W1': ('R1W1', 'R1W8'), 'R2W2': ('R1W2', 'R1W7'), 'R2W3': ('R1W3', 'R1W6'), 'R2W4': ('R1W4', 'R1W5'), 'R2X1': ('R1X1', 'R1X8'), 'R2X2': ('R1X2', 'R1X7'), 'R2X3': ('R1X3', 'R1X6'), 'R2X4': ('R1X4', 'R1X5'), 'R2Y1': ('R1Y1', 'R1Y8'), 'R2Y2': ('R1Y2', 'R1Y7'), 'R2Y3': ('R1Y3', 'R1Y6'), 'R2Y4': ('R1Y4', 'R1Y5'), 'R2Z1': ('R1Z1', 'R1Z8'), 'R2Z2': ('R1Z2', 'R1Z7'), 'R2Z3': ('R1Z3', 'R1Z6'), 'R2Z4': ('R1Z4', 'R1Z5'), 'R3W1': ('R2W1', 'R2W4'), 'R3W2': ('R2W2', 'R2W3'), 'R3X1': ('R2X1', 'R2X4'), 'R3X2': ('R2X2', 'R2X3'), 'R3Y1': ('R2Y1', 'R2Y4'), 'R3Y2': ('R2Y2', 'R2Y3'), 'R3Z1': ('R2Z1', 'R2Z4'), 'R3Z2': ('R2Z2', 'R2Z3'), 'R4W1': ('R3W1', 'R3W2'), 'R4X1': ('R3X1', 'R3X2'), 'R4Y1': ('R3Y1', 'R3Y2'), 'R4Z1': ('R3Z1', 'R3Z2'), 'R5WX': ('R4W1', 'R4X1'), 'R5YZ': ('R4Y1', 'R4Z1'), 'R6CH': ('R5WX', 'R5YZ')}


def get_play_in(results):
    play_in = set()
    for team1, team2 in results:
        if len(team1) > 3:
            play_in.add(team1[:3])
        if len(team2) > 3:
            play_in.add(team2[:3])
    return list(play_in)


def slots_with_play_in(play_in):
    out = SLOTS.copy()
    for slot in play_in:
        out[slot] = f'{slot}a', f'{slot}b'
    return out


def build_bracket_dict(results_dict):
    bracket = {}
    play_in = get_play_in(results_dict)
    slots = slots_with_play_in(play_in)
    levels = [play_in] + LEVELS
    # Play-in phase
    for key in levels[0]:
        bracket[key] = results_dict[slots[key]]
    # Regular phase
    for i in range(1, 7):
        for key in levels[i]:
            k1, k2 = slots[key]
            if k1 in levels[i-1]:
                k1 = bracket[k1]
            if k2 in levels[i-1]:
                k2 = bracket[k2]
            bracket[key] = results_dict[(k1, k2)]
    return bracket


def build_bracket_tab(results_tab, seeds_tab):
    seeds_dict = seeds_to_dict(seeds_tab)
    results_dict = results_to_dict(results_tab, seeds_dict)
    bracket_dict = build_bracket_dict(results_dict)
    # Remove play-in phase, map W16a -> W16, ...
    data = {'Slot': [], 'Team': []}
    for slot, win in bracket_dict.items():
        if not slot.startswith('R'):
            continue
        data['Slot'].append(slot)
        data['Team'].append(win[:3])
    return pl.DataFrame(data)

In [4]:
m_results = TABS['MNCAATourneyCompactResults'].filter(pl.col('Season') == 2023)
w_results = TABS['WNCAATourneyCompactResults'].filter(pl.col('Season') == 2023)
m_seeds = TABS['MNCAATourneySeeds'].filter(pl.col('Season') == 2023)
w_seeds = TABS['WNCAATourneySeeds'].filter(pl.col('Season') == 2023)
m_tab = build_bracket_tab(m_results, m_seeds)
w_tab = build_bracket_tab(w_results, w_seeds)

In [5]:
submission = pl.concat([
    m_tab.with_columns(
        pl.lit('M').alias('Tournament'),
        pl.lit(1).alias('Bracket')),
    w_tab.with_columns(
        pl.lit('W').alias('Tournament'),
        pl.lit(1).alias('Bracket')),
]).with_row_index(name='RowId')[['RowId', 'Tournament', 'Bracket', 'Slot', 'Team']]
submission.write_csv('submission.csv')